In [ ]:
# Install Miniconda3
!pip install -q condacolab
import condacolab
condacolab.install_miniconda(prefix='/usr/local')
print('The Miniconda has been installed...')

In [ ]:
import os
os.chdir("/kaggle/working/")
CODE_DIR = "InvSR"
print("Done...")

In [ ]:
# Clone Repository
import shutil
from pathlib import Path
if Path(CODE_DIR).exists():
  shutil.rmtree(CODE_DIR)

!git clone https://github.com/zsyOAOA/InvSR.git $CODE_DIR
os.chdir(f"./{CODE_DIR}")
print("Done...")

In [ ]:
"""
Managing environments Anaconda:
https://docs.conda.io/projects/conda/en/4.6.0/user-guide/tasks/manage-environments.html
"""
# Viewing the list of your environments
!conda info --envs
# Updating the environment.
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -U xformers==0.0.27.post2 --index-url https://download.pytorch.org/whl/cu121
!pip install -e ".[torch]"
!pip install -r requirements.txt
print("The environemnt has beed updated...")

In [4]:
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Zoomed LR', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('InvSR', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  if img.ndim > 3:
    img = img[:, :, :3]
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

In [ ]:
#  Real-World Image super-resolution with one steps
!python inference_invsr.py -i testdata/RealSet80/29.jpg -o outputs/RealSet80_one --num_steps 1 --chopping_size 128

im_lq = imread("testdata/RealSet80/29.jpg")
im_hq = imread("outputs/RealSet80_one/29.png")
display(im_lq, im_hq)


In [ ]:
# Starting the Web UI with pinggy
!conda install openssh -y

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/InvSR/app.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7860 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()